# TransE的pytorch实现

TransE不是神经网络模型，它无法理解句子的含义，因而得分不高，随便跑跑mrr大概在42左右，调下参大概有45-50。

TransE的优势在于不吃算力，仅靠CPU就能运行，这里我写的GPU版本速度更快。而且代码简单。

又试了一下，我已经复现不出自己的得分了，太玄学了，大概策略就是先norm取1得到最优模型，加载该模型并norm改为2训练得到最优模型，再加载该模型norm取3得到最优模型，再反复调整后最优分数就是50左右；
在算力允许的情况下，batchsize尽可能大，比如10万-150万（就是全部）；然后就是embedding维度我感觉影响不大，100维包含的信息足够丰富了，试过256和512，貌似大一点有时要好一些，不太确定，太大了会比较费算力；学习率粗调可以在0.01-0.001之间，微调就在0.001-0.0001之间吧。

提供给大家学习，代码也可以在我的github下载：https://github.com/renqi1/TransE_Pytorch_OpenBG500


In [1]:
import torch
from torch import nn
from torch.utils import data
import numpy as np
import tqdm

## 构建数据集

In [2]:
# 训练集和验证集
class TripleDataset(data.Dataset):
    def __init__(self, ent2id, rel2id, triple_data_list):
        self.ent2id = ent2id
        self.rel2id = rel2id
        self.data = triple_data_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        head, relation, tail = self.data[index]
        head_id = self.ent2id[head]
        relation_id = self.rel2id[relation]
        tail_id = self.ent2id[tail]
        return head_id, relation_id, tail_id

# 测试集    
class TestDataset(data.Dataset):
    def __init__(self, ent2id, rel2id, test_data_list):
        self.ent2id = ent2id
        self.rel2id = rel2id
        self.data = test_data_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        head, relation = self.data[index]
        head_id = self.ent2id[head]
        relation_id = self.rel2id[relation]
        return head_id, relation_id

## TransE模型

In [3]:
class TransE(nn.Module):

    def __init__(self, entity_num, relation_num, norm=1, dim=100):
        super(TransE, self).__init__()
        self.norm = norm
        self.dim = dim
        self.entity_num = entity_num
        self.entities_emb = self._init_emb(entity_num)
        self.relations_emb = self._init_emb(relation_num)

    def _init_emb(self, num_embeddings):
        embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=self.dim)
        uniform_range = 6 / np.sqrt(self.dim)
        embedding.weight.data.uniform_(-uniform_range, uniform_range)
        embedding.weight.data = torch.div(embedding.weight.data, embedding.weight.data.norm(p=2, dim=1, keepdim=True))
        return embedding

    def forward(self, positive_triplets: torch.LongTensor, negative_triplets: torch.LongTensor):
        positive_distances = self._distance(positive_triplets)
        negative_distances = self._distance(negative_triplets)
        return positive_distances, negative_distances

    def _distance(self, triplets):
        heads = self.entities_emb(triplets[:, 0])
        relations = self.relations_emb(triplets[:, 1])
        tails = self.entities_emb(triplets[:, 2])
        return (heads + relations - tails).norm(p=self.norm, dim=1)

    def link_predict(self, head, relation, tail=None, k=10):
        # h_add_r: [batch size, embed size] -> [batch size, 1, embed size] -> [batch size, entity num, embed size]
        h_add_r = self.entities_emb(head) + self.relations_emb(relation)
        h_add_r = torch.unsqueeze(h_add_r, dim=1)
        h_add_r = h_add_r.expand(h_add_r.shape[0], self.entity_num, self.dim)
        # embed_tail: [batch size, embed size] -> [batch size, entity num, embed size]
        embed_tail = self.entities_emb.weight.data.expand(h_add_r.shape[0], self.entity_num, self.dim)
        # values: [batch size, k] scores, the smaller, the better
        # indices: [batch size, k] indices of entities ranked by scores
        values, indices = torch.topk(torch.norm(h_add_r - embed_tail, dim=2), k=self.entity_num, dim=1, largest=False)
        if tail is not None:
            tail = tail.view(-1, 1)
            rank_num = torch.eq(indices, tail).nonzero().permute(1, 0)[1]+1
            rank_num[rank_num > 9] = 10000
            mrr = torch.sum(1/rank_num)
            hits_1_num = torch.sum(torch.eq(indices[:, :1], tail)).item()
            hits_3_num = torch.sum(torch.eq(indices[:, :3], tail)).item()
            hits_10_num = torch.sum(torch.eq(indices[:, :10], tail)).item()
            return mrr, hits_1_num, hits_3_num, hits_10_num     # 返回一个batchsize, mrr的和，hit@k的和
        return indices[:, :k]

    def evaluate(self, data_loader, dev_num=5000.0):
        mrr_sum = hits_1_nums = hits_3_nums = hits_10_nums = 0
        for heads, relations, tails in tqdm.tqdm(data_loader):
            mrr_sum_batch, hits_1_num, hits_3_num, hits_10_num = self.link_predict(heads.cuda(), relations.cuda(), tails.cuda())
            mrr_sum += mrr_sum_batch
            hits_1_nums += hits_1_num
            hits_3_nums += hits_3_num
            hits_10_nums += hits_10_num
        return mrr_sum/dev_num, hits_1_nums/dev_num, hits_3_nums/dev_num, hits_10_nums/dev_num

## 设置参数

In [4]:
# batchsize增大，得分略有上升
train_batch_size = 100000
dev_batch_size = 20  # 显存不够就调小
test_batch_size = 20
epochs = 40
margin = 1
print_frequency = 5  # 每多少step输出一次信息
validation = True  # 是否验证，验证比较费时
dev_interval = 5  # 每多少轮验证一次，微调设小一点，会保存最佳权重
best_mrr = 0
learning_rate = 0.001  # 学习率建议粗调0.01-0.001，精调0.001-0.0001
distance_norm = 3  # 论文是L1距离效果不好，取2或3效果好
embedding_dim = 100  # 维度增大可能会有提升，我感觉没用，100维包含的信息足够丰富

## 加载数据集

In [7]:
path = 'data/OpenBG500/OpenBG500_'

In [8]:
with open(path+'entity2text.tsv', 'r', encoding='utf-8') as fp:
    dat = fp.readlines()
    lines = [line.strip('\n').split('\t') for line in dat]
ent2id = {line[0]: i for i, line in enumerate(lines)}
id2ent = {i: line[0] for i, line in enumerate(lines)}
with open(path+'relation2text.tsv', 'r', encoding='utf-8') as fp:
    dat = fp.readlines()
    lines = [line.strip().split('\t') for line in dat]
rel2id = {line[0]: i for i, line in enumerate(lines)}
with open(path+'train.tsv', 'r', encoding='utf-8') as fp:
    dat = fp.readlines()
    train = [line.strip('\n').split('\t') for line in dat]
with open(path+'dev.tsv', 'r', encoding='utf-8') as fp:
    dat = fp.readlines()
    dev = [line.strip('\n').split('\t') for line in dat]
with open(path+'test.tsv', 'r', encoding='utf-8') as fp:
    test = fp.readlines()
    test = [line.strip('\n').split('\t') for line in test]
# 构建数据集
train_dataset = TripleDataset(ent2id, rel2id, train)
dev_dataset = TripleDataset(ent2id, rel2id, dev)
train_data_loader = data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
dev_data_loader = data.DataLoader(dev_dataset, batch_size=dev_batch_size)
test_dataset = TestDataset(ent2id, rel2id, test)
test_data_loader = data.DataLoader(test_dataset, batch_size=test_batch_size)

## 训练和验证

In [9]:
# 构建模型
model = TransE(len(ent2id), len(rel2id), norm=distance_norm, dim=embedding_dim).cuda()
# model.load_state_dict(torch.load('transE_best.pth'))
# 优化器adam
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# 损失函数， 对于本例，loss=max(0, (pd-nd)+1)， 负样本距离越小，正样本距离越大越好
criterion = nn.MarginRankingLoss(margin=margin, reduction='mean')
# 开始训练
print('start training...')
for epoch in range(epochs):
    all_loss = 0
    for i, (local_heads, local_relations, local_tails) in enumerate(train_data_loader):

        positive_triples = torch.stack((local_heads, local_relations, local_tails), dim=1).cuda()

        # 生成负样本
        head_or_tail = torch.randint(high=2, size=local_heads.size())
        random_entities = torch.randint(high=len(ent2id), size=local_heads.size())
        broken_heads = torch.where(head_or_tail == 1, random_entities, local_heads)
        broken_tails = torch.where(head_or_tail == 0, random_entities, local_tails)
        negative_triples = torch.stack((broken_heads, local_relations, broken_tails), dim=1).cuda()

        # # 生成负样本, 只打乱tail
        # random_entities = torch.randint(high=len(ent2id), size=local_heads.size())
        # negative_triples = torch.stack((random_entities, local_relations, random_entities), dim=1).cuda()

        optimizer.zero_grad()
        pd, nd = model(positive_triples, negative_triples)
        # pd要尽可能小， nd要尽可能大
        loss = criterion(pd, nd, torch.tensor([-1], dtype=torch.long).cuda())
        loss.backward()
        all_loss += loss.data
        optimizer.step()
        if i % print_frequency == 0:
            print(
                f"epoch:{epoch}/{epochs}, step:{i}/{len(train_data_loader)}, loss={loss.item()}, avg_loss={all_loss / (i + 1)}")
    print(f"epoch:{epoch}/{epochs}, all_loss={all_loss}")

    # 验证
    if validation and (epoch + 1) % dev_interval == 0:
        print('testing...')
        improve = ''
        mrr, hits1, hits3, hits10 = model.evaluate(dev_data_loader)
        if mrr >= best_mrr:
            best_mrr = mrr
            improve = '*'
            torch.save(model.state_dict(), 'transE_best.pth')
        torch.save(model.state_dict(), 'transE_latest.pth')
        print(f'mrr: {mrr}, hit@1: {hits1}, hit@3: {hits3}, hit@10: {hits10}  {improve}')
    if not validation:
        torch.save(model.state_dict(), 'transE_latest.pth')

start training...
epoch:0/40, step:0/13, loss=0.9999334812164307, avg_loss=0.9999334812164307
epoch:0/40, step:5/13, loss=0.9918926954269409, avg_loss=0.9959738254547119
epoch:0/40, step:10/13, loss=0.9838432669639587, avg_loss=0.9918791651725769
epoch:0/40, all_loss=12.872831344604492
epoch:1/40, step:0/13, loss=0.975989818572998, avg_loss=0.975989818572998
epoch:1/40, step:5/13, loss=0.9672017097473145, avg_loss=0.971381664276123
epoch:1/40, step:10/13, loss=0.9584214687347412, avg_loss=0.9669575691223145
epoch:1/40, all_loss=12.548501968383789
epoch:2/40, step:0/13, loss=0.9518741965293884, avg_loss=0.9518741965293884
epoch:2/40, step:5/13, loss=0.9433963298797607, avg_loss=0.9473958015441895
epoch:2/40, step:10/13, loss=0.9349619150161743, avg_loss=0.9431370496749878
epoch:2/40, all_loss=12.238751411437988
epoch:3/40, step:0/13, loss=0.9285838603973389, avg_loss=0.9285838603973389
epoch:3/40, step:5/13, loss=0.9198927283287048, avg_loss=0.9241694808006287
epoch:3/40, step:10/13, lo

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:20<00:00, 12.20it/s]


mrr: 0.21487672626972198, hit@1: 0.0212, hit@3: 0.3926, hit@10: 0.5074  *
epoch:5/40, step:0/13, loss=0.882912814617157, avg_loss=0.882912814617157
epoch:5/40, step:5/13, loss=0.8743348121643066, avg_loss=0.878745973110199
epoch:5/40, step:10/13, loss=0.8665673136711121, avg_loss=0.8746995329856873
epoch:5/40, all_loss=11.350520133972168
epoch:6/40, step:0/13, loss=0.8611227869987488, avg_loss=0.8611227869987488
epoch:6/40, step:5/13, loss=0.8519725799560547, avg_loss=0.8562087416648865
epoch:6/40, step:10/13, loss=0.8446199893951416, avg_loss=0.852358341217041
epoch:6/40, all_loss=11.061125755310059
epoch:7/40, step:0/13, loss=0.8380495309829712, avg_loss=0.8380495309829712
epoch:7/40, step:5/13, loss=0.8301265239715576, avg_loss=0.8341266512870789
epoch:7/40, step:10/13, loss=0.822293758392334, avg_loss=0.8300257921218872
epoch:7/40, all_loss=10.769623756408691
epoch:8/40, step:0/13, loss=0.815977931022644, avg_loss=0.815977931022644
epoch:8/40, step:5/13, loss=0.8072604537010193, av

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:21<00:00, 11.88it/s]


mrr: 0.449470579624176, hit@1: 0.3204, hit@3: 0.5494, hit@10: 0.7  *
epoch:10/40, step:0/13, loss=0.7695481777191162, avg_loss=0.7695481777191162
epoch:10/40, step:5/13, loss=0.7612093687057495, avg_loss=0.765253484249115
epoch:10/40, step:10/13, loss=0.7537643313407898, avg_loss=0.7611345052719116
epoch:10/40, all_loss=9.8736572265625
epoch:11/40, step:0/13, loss=0.7454591989517212, avg_loss=0.7454591989517212
epoch:11/40, step:5/13, loss=0.7374083995819092, avg_loss=0.7416588664054871
epoch:11/40, step:10/13, loss=0.7311813235282898, avg_loss=0.7375125288963318
epoch:11/40, all_loss=9.56297492980957
epoch:12/40, step:0/13, loss=0.7222965359687805, avg_loss=0.7222965359687805
epoch:12/40, step:5/13, loss=0.7124143838882446, avg_loss=0.7172896265983582
epoch:12/40, step:10/13, loss=0.7041597366333008, avg_loss=0.7130352854728699
epoch:12/40, all_loss=9.246464729309082
epoch:13/40, step:0/13, loss=0.6980627179145813, avg_loss=0.6980627179145813
epoch:13/40, step:5/13, loss=0.68827515840

...

mrr: 0.5408957004547119, hit@1: 0.4288, hit@3: 0.6244, hit@10: 0.7728  *
epoch:30/40, step:0/13, loss=0.3563702702522278, avg_loss=0.3563702702522278
epoch:30/40, step:5/13, loss=0.35200655460357666, avg_loss=0.35402044653892517
epoch:30/40, step:10/13, loss=0.3483053147792816, avg_loss=0.3518577516078949
epoch:30/40, all_loss=4.561625957489014
epoch:31/40, step:0/13, loss=0.343534916639328, avg_loss=0.343534916639328
epoch:31/40, step:5/13, loss=0.33963093161582947, avg_loss=0.3425942659378052
epoch:31/40, step:10/13, loss=0.3365355432033539, avg_loss=0.3408218324184418
epoch:31/40, all_loss=4.423052787780762
epoch:32/40, step:0/13, loss=0.33443889021873474, avg_loss=0.33443889021873474
epoch:32/40, step:5/13, loss=0.33007538318634033, avg_loss=0.3324008285999298
epoch:32/40, step:10/13, loss=0.3284552991390228, avg_loss=0.33042535185813904
epoch:32/40, all_loss=4.285455226898193
epoch:33/40, step:0/13, loss=0.32367369532585144, avg_loss=0.32367369532585144
epoch:33/40, step:5/13, los

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:20<00:00, 11.92it/s]


mrr: 0.5411531925201416, hit@1: 0.4302, hit@3: 0.6214, hit@10: 0.7734  *
epoch:35/40, step:0/13, loss=0.3080292344093323, avg_loss=0.3080292344093323
epoch:35/40, step:5/13, loss=0.304004043340683, avg_loss=0.30543971061706543
epoch:35/40, step:10/13, loss=0.3030575215816498, avg_loss=0.3043721318244934
epoch:35/40, all_loss=3.950467586517334
epoch:36/40, step:0/13, loss=0.3025049567222595, avg_loss=0.3025049567222595
epoch:36/40, step:5/13, loss=0.2963460087776184, avg_loss=0.29817861318588257
epoch:36/40, step:10/13, loss=0.2952970266342163, avg_loss=0.29706981778144836
epoch:36/40, all_loss=3.856097936630249
epoch:37/40, step:0/13, loss=0.2929033637046814, avg_loss=0.2929033637046814
epoch:37/40, step:5/13, loss=0.2895614504814148, avg_loss=0.29128414392471313
epoch:37/40, step:10/13, loss=0.2882974147796631, avg_loss=0.2905087471008301
epoch:37/40, all_loss=3.7714219093322754
epoch:38/40, step:0/13, loss=0.2857908606529236, avg_loss=0.2857908606529236
epoch:38/40, step:5/13, loss=0

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:21<00:00, 11.90it/s]


mrr: 0.5394382476806641, hit@1: 0.427, hit@3: 0.6212, hit@10: 0.776  


## 预测

In [10]:
predict_all = []
model.load_state_dict(torch.load('transE_best.pth'))
for heads, relations in tqdm.tqdm(test_data_loader):
    # 预测的id,结果为tensor(batch_size*10)
    predict_id = model.link_predict(heads.cuda(), relations.cuda())
    # 结果取到cpu并转为一行的list以便迭代
    predict_list = predict_id.cpu().numpy().reshape(1,-1).squeeze(0).tolist()
    # id转为实体
    predict_ent = map(lambda x: id2ent[x], predict_list)
    # 保存结果
    predict_all.extend(predict_ent)
print('prediction finished !')

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:20<00:00, 12.15it/s]

prediction finished !


## 写入文件并保存

In [11]:
# 写入文件，按提交要求
with open('submission.tsv', 'w', encoding='utf-8') as f:
    for i in range(len(test)):
        # 直接writelines没有空格分隔，手工加分割符，得按提交格式来
        list = [x + '\t' for x in test[i]] + [x + '\n' if i == 9 else x + '\t' for i, x in enumerate(predict_all[i*10:i*10+10])]
        f.writelines(list)
print('file saved !')

file saved !
